# **7. Optimizing Model Parameters**

모델과 데이터를 준비했으니 이제는 모델을 학습, 검증, 테스트 하면서 파라미터를 최적화할 차례임

모델 학습은 반복적인 과정임 각 반복에서 모델은 출력값을 추측하고, 그 추측이 얼마나 틀렸는지 loss로 계산

그다음 오차에 대해 파라미터별로 얼마나 영향을 미쳤는지 gradient를 계산하고 이를 통해 파라미터를 업데이트하는 것



In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512,10),
        )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 230kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 4.23MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 11.2MB/s]


In [ ]:
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

# Hyperparameters

하이퍼파라미터는 모델 최적화 과정에 영향을 주는 설정값. 값에 따라 학습 속도와 성능이 달라지므로 잘 조정해야됨

- epoch : 데이터셋을 몇번 반복할지
- batch size : 한번에 모델에 넣을 데이터 개수
- learning rate : 파라미터를 얼마나 빠르게 업데이트할

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Optimization loop

모델 학습은 두가지 과정을 반복하면서 이루어짐
1. train loop (훈련 루프) : 학습 데이터를 사용하여 모델 파라미터를 업데이트
2. test loop (검증 루프) : 테스트 데이터를 사용하여 모델 성능을 측정



# Loss Function

손실 함수는 예측값과 실제 정답의 차이를 계산함

분류 문제는 주로 nn.CrossEntropyLoss() 를 주로 사

In [ ]:
loss_fn = nn.CrossEntropyLoss()

# Optimizer

손실을 줄이기 위해 모델 파라미터를 업데이트하는 알고리즘

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# Full Implementation

In [ ]:
# 훈련 루프
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    pred = model(X)
    loss = loss_fn(pred, y)

    # optimization 하는 3단
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      print(f"loss: {loss.item():>7} [{batch * batch_size + len(X):>5d}/{size:>5}]")

# 테스트 루프
def test_loop(dataloader, model, loss_fn):
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
  print(f"EPOCH {t+1}\n---------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("끝")

EPOCH 1
---------------------
loss: 2.320664882659912 [   64/60000]
loss: 2.2948029041290283 [ 6464/60000]
loss: 2.28288197517395 [12864/60000]
loss: 2.2696921825408936 [19264/60000]
loss: 2.252969741821289 [25664/60000]
loss: 2.238802433013916 [32064/60000]
loss: 2.2357337474823 [38464/60000]
loss: 2.2115561962127686 [44864/60000]
loss: 2.2148666381835938 [51264/60000]
loss: 2.1782004833221436 [57664/60000]
test error: 
 Accuracy: 50.7%, Avg loss: 2.173300 

EPOCH 2
---------------------
loss: 2.197521209716797 [   64/60000]
loss: 2.171201229095459 [ 6464/60000]
loss: 2.1233208179473877 [12864/60000]
loss: 2.1329259872436523 [19264/60000]
loss: 2.085287570953369 [25664/60000]
loss: 2.0416107177734375 [32064/60000]
loss: 2.0599353313446045 [38464/60000]
loss: 1.9914841651916504 [44864/60000]
loss: 1.9989672899246216 [51264/60000]
loss: 1.9290918111801147 [57664/60000]
test error: 
 Accuracy: 62.3%, Avg loss: 1.923103 

EPOCH 3
---------------------
loss: 1.9623866081237793 [   64/60000